In [1]:
path_root = r"C:/Users/jgpg000.edu/Desktop/Agente inteligente recuperación sensórica IoT/"

# ruta clases personalizadas
import sys
path_classes = f"{path_root}classes"
sys.path.append(path_classes)

## LIBRERIAS
import pandas as pd
from class_data import *
from fun_models import *

c:\Users\jgpg000.edu\AppData\Local\anaconda3\envs\management\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Agente inteligente

#### Cargar base de datos

In [2]:
path_excel = f"{path_root}datasets/database_items/item_list.xlsx"
path_saved_json = f"{path_root}datasets/database_items/database_json/"

documentos = load_documents(path_excel, path_saved_json)
documentos_langchain = load_documents_langchain(documentos)

from langchain_text_splitters import RecursiveCharacterTextSplitter
# Aplicamos el splitter de Langchain
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function=len,
    is_separator_regex=False,
)

# Dividimos cada texto en chunks
chunks_texto = text_splitter.split_documents(documentos_langchain)
print(f"Hay {len(chunks_texto)} chunks de texto.")

100%|██████████| 88/88 [00:00<00:00, 222.43it/s]


Se han cargado 86 documentos.


100%|██████████| 86/86 [00:00<00:00, 7641.35it/s]


Se han generado 954 documentos de Langchain.
    De ellos, 86 son el texto y 868 son resúmenes de tablas.
Hay 7218 chunks de texto.


#### Modelo RAG

In [3]:
path_saved_vector = f"{path_root}datasets/database_items/database_vectorial/"
name_database = "large_esp"
model_llm = "gpt-4o"

template_sistema = """ Eres un asistente de una empresa encargada de smart cities.
Como preguntas, te van a adjuntar listas de requisitos de equipo para proyecto.
Tu labor es responder con el nombre de los equipos que encuentres que cumplan los requisitos, así como suministrar la información de estos dispositivos en relación a los requisitos.
Usa la información añadida en el contexto para responder a la pregunta pero no hagas referencias al mismo.
Se añade como metadato tanto el proveedor como el nombre a los que pertenece el contexto devuelto.
Responde únicamente con la especificación que se pide. Si no aparece di que no lo sabes.
Pregunta: {question}
Metadato: {metadata}
Contexto: {contexts}
La respuesta debe tener el siguiente formato:
- Nombre del equipo y proveedor.
- Especificaciones que se han preguntado y aparecen en el contexto.
Respuesta:
"""

vectorstore = create_database(f"{path_saved_vector}{name_database}",
                               chunks_texto, nombre_embedding = "text-embedding-3-large")
retriever = create_retriever(vectorstore)
rag = create_RAG(retriever=retriever,
                plantilla_RAG=template_sistema,
                model=model_llm)


#### Añadimos filtro

In [4]:
rag_filtrado = rag_filter(vectorstore, 
                        equipo="router",
                        template_system=template_sistema,
                        model_llm=model_llm)

response = rag_filtrado.invoke("Necesito un router con 4 puertos ethernet y 2 puertos USB.")
response["answer"]

'- Nombre del equipo y proveedor: MikroTik hEX PoE, MikroTik.\n- Especificaciones: 5 puertos Ethernet (de los cuales 4 pueden alimentar otros dispositivos), 1 puerto USB 2.0.'

#### Interfaz gráfica

In [ ]:
import gradio as gr

def predict(message, history):
    response = rag_filtrado.invoke(message)
    return response['answer']

gr.ChatInterface(predict).launch()